## Классификация клиетов телеком компании

### Описание исследования
Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

### Цель исследования
На основе данных предложить клиенту тариф.

### Задачи исследования
1. Изучение файла
2. Разбить данные на выборки
3. Иссдедовать модели
4. Проверить модель на тестовой выборке
5. Проверить модель на адекватность

### Исходные данные
   - `сalls` — количество звонков,
   - `minutes` — суммарная длительность звонков в минутах,
   - `messages` — количество sms-сообщений,
   - `mb_used` — израсходованный интернет-трафик в Мб,
   - `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

In [2]:
try:
    data = pd.read_csv('/datasets/users_behavior.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/users_behavior.csv')
    
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


## Разбейте данные на выборки

Применим функцию `train_test_split` дважды, чтобы разделить нашу таблицу  в соотношении 3:1:1. На обучающую, валидационную и тестовую

In [4]:
data_train, data2 = train_test_split(
    data, test_size=0.4, random_state=12345, stratify=data['is_ultra'])

In [5]:
#проверим размеры наших таблиц
display(data_train.shape)
display(data2.shape)


(1928, 5)

(1286, 5)

In [6]:
data_valid, data_test = train_test_split(
    data2, test_size=0.5, random_state=12345, stratify=data2['is_ultra'])

In [7]:
#перепроверим
display(data_valid.shape)
display(data_test.shape)


(643, 5)

(643, 5)

In [8]:
#Разобьем наши данные по признакам и целевому объекту. для тренирочной таблицы
features_train = data_train.drop(['is_ultra'], axis=1)
target_train = data_train['is_ultra']

In [9]:
#для проверочной таблицы
features_valid = data_valid.drop(['is_ultra'], axis=1)
target_valid = data_valid['is_ultra']

In [10]:
#для тестовой таблицы
features_test = data_test.drop(['is_ultra'], axis=1)
target_test = data_test['is_ultra']

## Исследуйте модели

Начнем с модели "дерево".
Переберем в цикле от 1 до 5 максимальную глубину. Рассчитаем `accuracy`, и присвоим для модели такую глубину дерева с максимальным `accuracy`


In [11]:
best_model_tree = None
best_result = 0
for i in range(1, 11):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=i)
    model_tree.fit(features_train, target_train)
    predictions_valid = model_tree.predict(features_valid)
    result = accuracy_score(target_valid, predictions_valid)
    print('max_depth =', i, ': ', end='')
    print(result)
    if result > best_result:
        best_model_tree = model_tree
        best_result = result 
print("Accuracy наилучшей модели на валидационной выборке:", best_result)

max_depth = 1 : 0.7402799377916018
max_depth = 2 : 0.7729393468118196
max_depth = 3 : 0.7776049766718507
max_depth = 4 : 0.7542768273716952
max_depth = 5 : 0.7853810264385692
max_depth = 6 : 0.7744945567651633
max_depth = 7 : 0.7869362363919129
max_depth = 8 : 0.80248833592535
max_depth = 9 : 0.7822706065318819
max_depth = 10 : 0.7729393468118196
Accuracy наилучшей модели на валидационной выборке: 0.80248833592535


Модель "случайный лес". Переберем в цикле количество деревьев в лесу от 1 до 10. Также рассчитаем `accuracy`, и присвоим для модели такое значение `n_estimators` где `accuracy` максимально

In [13]:
best_model_forest = None
best_result = 0
for est in range(10, 101, 10):
    for depth in range (1, 11):
        model_forest = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth) 
        model_forest.fit(features_train, target_train) 
        result = model_forest.score(features_valid, target_valid) 
        if result > best_result:
            best_model_forest = model_forest
            best_result = result 

print("Accuracy наилучшей модели на валидационной выборке:", best_result)

Accuracy наилучшей модели на валидационной выборке: 0.8211508553654744


Логистическая регрессия. Пройдемся циклом по параметрам  `solver'`, чтобы найти лучшее решение.

In [14]:
best_model_logistic = None
best_result = 0
solvers = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
for i in solvers:
    model_logistic = LogisticRegression(random_state=12345, solver=i, max_iter=1000)
    model_logistic.fit(features_train, target_train)
    result = model_logistic.score(features_valid, target_valid)
    print(i, result)
    if result > best_result:
        best_model_logistic = model_logistic
        best_result = result 

print("Accuracy наилучшей модели на валидационной выборке:", best_result)
    

C:\Users\Mi\anaconda3\Lib\site-packages\scipy\optimize\_linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\Mi\anaconda3\Lib\site-packages\scipy\optimize\_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)


newton-cg 0.7387247278382582
lbfgs 0.7387247278382582
liblinear 0.71850699844479


C:\Users\Mi\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


sag 0.702954898911353
saga 0.6936236391912908
Accuracy наилучшей модели на валидационной выборке: 0.7387247278382582


C:\Users\Mi\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Результаты `accuracy` на валидационной выборке оказались выше у модели "Случайный лес" - 0.82

## Проверьте модель на тестовой выборке

In [15]:
#проверим модель "случайный лес" на тестовой выборке
best_model_forest.score(features_test, target_test) 

0.8087091757387247

На тестовой выборке модель показывает точность близкую к валидационной. Это означает отсутствие проблем с переобучение модели.

## Проверьте модели на адекватность

In [16]:
dummy = DummyClassifier(strategy='most_frequent', random_state=123).fit(features_train, target_train)
print('Accuracy Dummy-модели на тестовой выборке:', accuracy_score(target_test, dummy.predict(features_test)))

Accuracy Dummy-модели на тестовой выборке: 0.6936236391912908


> Значения accuracy модели DummyClassifier (0.69) ниже, чем у выбранной нами модели (0.809), значит наша модель адекватна и эффективна.

## Вывод

1.Проведено исследование с целью разработки модели, позволяющей наиболее точно предложить пользователям мобильной связи тарифы «Смарт» или «Ультра».

2.Модель построена на основании поведения клиентов, уже перешедших на эти тарифы.

3.Были построены 3 модели:

'дерево решений' ( Decision Tree Classifier ),
'случайный лес' ( RandomForestClassifier ),
'логистическая регрессия' ( Logistic Regression ).
4.По результатам сравнения была выбрана лучшая - 'случайный лес'. Значение accuracy = 0.82.

5.Все модели соответствуют требованию по качеству, согласно которому значение accuracy должно быть не менее 0.75.

6.Лучшая модель проверена на тестовых объектах, не участвующих в обучении и валидации модели. Значение accuracy = 0.809.

7.Для проверки на адекватность выбранной нами модели 'случайный лес' ( RandomForestClassifier ) использовали модель DummyClassifier, которая показала результат accuracy хуже, чем результат нашей лучшей модели.

8.Результаты исследования позволят специалистам выбрать наиболее качественную модель, которая подберет клиентам новый тариф.